## Bienvenidxs al taller de DNS.

A lo largo del taller van a poder probar los diferentes comandos que nos provee la librería Scapy para realizar consultas DNS. El objetivo es que al finalizar el taller tengan el código necesario para poder hacer los experimentos y el informe.

El proceso de resolución de nombres consiste en sucesivas consultas y respuestas por parte de todos los servidores DNS involucrados. Las consultas suelen ser recursivas cuando las PC quieren resolver un nombre y le preguntan al Resolver local y suelen ser iterativas cuando los Resolvers le pasan las consultas a los servidores Autoritativos responsables de cada zona. Por esa razón, en una consulta determinada, puede haber subconsultas recursivas e iterativas. Además de los servidores autoritativos de cada zona, el sistema DNS no podría funcionar si no existieran
servidores por encima de toda la jerarquía de zonas que funcionen como punto de partida para comenzar las
consultas iterativas. Estos servidores se llaman Root Name Servers y tienen direcciones IP asignadas fijas,
que nunca cambian de manera que no haga falta hacer una consulta DNS para resolverlos porque sino no se
podría empezar. Estos servidores y sus direcciones IP están listados en la siguiente tabla:

Nombre del Servidor|Direcciones IP (IPv4, IPv6)|Entidad propietaria
:------------------|:--------------------------|:------------------
a.root-servers.net|198.41.0.4, 2001:503:ba3e::2:30|Verisign, Inc.
b.root-servers.net|199.9.14.201, 2001:500:200::b|University of Southern California
c.root-servers.net|192.33.4.12, 2001:500:2::c|Cogent Communications
d.root-servers.net|199.7.91.13, 2001:500:2d::d|University of Maryland
e.root-servers.net|192.203.230.10, 2001:500:a8::e|NASA (Ames Research Center)
f.root-servers.net|192.5.5.241, 2001:500:2f::f|Internet Systems Consortium, Inc.
g.root-servers.net|192.112.36.4, 2001:500:12::d0d|US Department of Defense (NIC)
h.root-servers.net|198.97.190.53, 2001:500:1::53|US Army (Research Lab)
i.root-servers.net|192.36.148.17, 2001:7fe::53|Netnod
j.root-servers.net|192.58.128.30, 2001:503:c27::2:30|Verisign, Inc.
k.root-servers.net|193.0.14.129, 2001:7fd::1|RIPE NCC
l.root-servers.net|199.7.83.42, 2001:500:9f::42|ICANN
m.root-servers.net|202.12.27.33, 2001:dc3::35|WIDE Project

### Utilizando Scapy consultar por el registro A de www.dc.uba.ar

In [1]:
from scapy.all import *

/home/nicolas/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/home/nicolas/.local/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [2]:
def getMailExchange(domain, full_crawl=False):
    dns = DNS(rd=1,qd=DNSQR(qname=domain, qtype="MX"))
    udp = UDP(sport=RandShort(), dport=53)
    ips = [
        ("198.41.0.4", 0),
        ("199.9.14.201", 0),
        ("192.33.4.12", 0),
        ("199.7.91.13", 0),
        ("192.203.230.10", 0),
        ("192.5.5.241", 0),
        ("192.112.36.4", 0),
        ("198.97.190.53", 0),
        ("192.36.148.17", 0),
        ("192.58.128.30", 0),
        ("193.0.14.129", 0),
        ("199.7.83.42", 0),
        ("202.12.27.33", 0)
    ]

    non_responsive_servers = set()
    authority_servers = set()
    amount_of_answers = 0
    
    while (len(ips) > 0):
        t = ips.pop()
        dst_ip, height = t[0], t[1]

        ip = IP(dst=dst_ip)
        answer = sr1(ip/udp/dns, verbose=0, timeout=10)
        
        if answer is not None and answer.haslayer(DNS):
            for i in range(answer[DNS].arcount):
                if answer[DNS].ar[i].type == 1:
                    authority_servers.add(dst_ip)

                    if answer[DNS].ancount == 0:
                        ips.append([answer[DNS].ar[i].rdata, height + 1])

            for i in range(answer[DNS].ancount):
                amount_of_answers += 1
                
                if not full_crawl:
                    print("Los DNS para el servidor de mail del dominio: {} - Altura: {} - Tipo: {} son: {}".format(
                        answer[DNS].an[i].rrname,
                        height,
                        answer[DNS].an[i].type,
                        [p.exchange for p in answer[DNS].an[i].iterpayloads()]
                    ))
                    ips = []
        else:
            non_responsive_servers.add(dst_ip)
    
    if full_crawl:
        print("#Authority Servers: {} - {}".format(len(authority_servers), authority_servers))
        print("#Authority Servers NO RESPONSE: {} - {}".format(len(non_responsive_servers), non_responsive_servers))
        print("#Responses: {}".format(amount_of_answers))


# Corridas

In [10]:
# Obtencion del Dominio MX
print("Universidad de Uzbekistan -- MX:")
getMailExchange("nuu.uz.", False)

print("Universidad de Polonia -- MX:")
getMailExchange("us.edu.pl.", False)

print("Universidad de Rusia -- MX:")
getMailExchange("www.hse.ru.", False)

Universidad de Uzbekistan -- MX:
Los DNS para el servidor de mail del dominio: b'nuu.uz.' - Altura: 2 - Tipo: 15 son: [b'mx.yandex.net.']
Universidad de Polonia -- MX:
Los DNS para el servidor de mail del dominio: b'us.edu.pl.' - Altura: 2 - Tipo: 15 son: [b'd69205b.ess.barracudanetworks.com.', b'd69205a.ess.barracudanetworks.com.']
Los DNS para el servidor de mail del dominio: b'us.edu.pl.' - Altura: 2 - Tipo: 15 son: [b'd69205a.ess.barracudanetworks.com.']
Universidad de Rusia -- MX:
Los DNS para el servidor de mail del dominio: b'www.hse.ru.' - Altura: 2 - Tipo: 5 son: [b'mg2.hse.ru.', b'mg2.hse.ru.', b'mg1.hse.ru.']
Los DNS para el servidor de mail del dominio: b'hse.ru.' - Altura: 2 - Tipo: 15 son: [b'mg2.hse.ru.', b'mg1.hse.ru.']
Los DNS para el servidor de mail del dominio: b'hse.ru.' - Altura: 2 - Tipo: 15 son: [b'mg1.hse.ru.']


In [11]:
# Obtencion del Dominio MX
print("Universidad de Uzbekistan -- Full Crawl")
getMailExchange("nuu.uz.", True)

print("Universidad de Polonia -- Full Crawl")
getMailExchange("us.edu.pl.", True)

print("Universidad de Rusia -- Full Crawl")
getMailExchange("www.hse.ru.", True)

Universidad de Uzbekistan -- Full Crawl
#Authority Servers: 21 - {'62.209.143.181', '192.112.36.4', '202.12.27.33', '185.74.6.29', '192.33.4.12', '193.0.14.129', '198.97.190.53', '192.36.148.17', '81.95.225.245', '217.12.81.129', '199.7.83.42', '192.58.128.30', '104.238.81.247', '199.9.14.201', '192.5.5.241', '83.69.129.33', '91.212.89.244', '185.196.212.7', '192.203.230.10', '198.41.0.4', '199.7.91.13'}
#Authority Servers NO RESPONSE: 0 - set()
#Responses: 182
Universidad de Polonia -- Full Crawl
#Authority Servers: 24 - {'185.159.197.48', '192.195.72.53', '192.112.36.4', '202.12.27.33', '192.33.4.12', '156.154.100.15', '193.0.14.129', '185.159.198.48', '194.0.25.29', '198.97.190.53', '149.156.1.252', '192.36.148.17', '93.190.128.146', '199.7.83.42', '46.28.245.82', '192.58.128.30', '199.9.14.201', '192.5.5.241', '155.158.99.2', '192.203.230.10', '198.41.0.4', '194.181.87.156', '199.7.91.13', '155.158.102.7'}
#Authority Servers NO RESPONSE: 0 - set()
#Responses: 448
Universidad de Rus

## Ejercicio:
Adaptar el código anterior de manera que, a través de sucesivas consultas iterativas se obtenga el registro
MX de un dominio dado. Para esto, tener en cuenta que en cada consulta DNS puede tener 3 tipos de respuestas: 
1. nos devuelven los servidores DNS a los cuales seguir preguntando
2. nos devuelven la respuesta a la consulta que estamos haciendo
3. nos devuelven el registro SOA de la zona indicando que el registro solicitado no forma parte de la base de datos de nombres de la zona.